In [2]:
import polars as pl
import polars.selectors as cs

In [3]:
disasters = pl.read_csv("1970-2021_DISASTERS.csv", ignore_errors=True).rename(
        {"Disaster Type" : "disasterType"})

disasters

Total death Columns: shape: (1, 1)
┌──────────────┐
│ Total Deaths │
│ ---          │
│ u32          │
╞══════════════╡
│ 10199        │
└──────────────┘
Total Damages Columns: shape: (1, 1)
┌──────────────────────────┐
│ Total Damages ('000 US$) │
│ ---                      │
│ u32                      │
╞══════════════════════════╡
│ 4863                     │
└──────────────────────────┘


Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,disasterType,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,i64,str,i64,str
"""1970-0013-ARG""",1970,13,null,"""Natural""","""Hydrological""","""Flood""",null,null,null,"""Argentina""","""ARG""","""South America""","""Americas""","""Mendoza""",null,null,null,null,null,null,null,null,"""Km2""",null,null,null,null,1970,1,4,1970,1,4,36,null,null,null,null,null,null,25000,15.001282,null,null,null,null
"""1970-0109-AUS""",1970,109,null,"""Natural""","""Meteorological""","""Storm""","""Tropical cyclone""",null,"""Ada""","""Australia""","""AUS""","""Australia and New Zealand""","""Oceania""","""Queensland""",null,null,null,null,null,null,null,null,"""Kph""",null,null,null,null,1970,1,null,1970,1,null,13,null,null,null,null,null,null,72475,15.001282,null,null,null,null
"""1970-0044-BEN""",1970,44,null,"""Natural""","""Hydrological""","""Flood""",null,null,null,"""Benin""","""BEN""","""Western Africa""","""Africa""","""Atacora region""",null,null,null,"""Yes""",null,null,null,null,"""Km2""",null,null,null,null,1970,9,null,1970,9,null,null,null,null,null,null,null,null,200,15.001282,null,null,null,null
"""1970-0063-BGD""",1970,63,null,"""Natural""","""Meteorological""","""Storm""","""Tropical cyclone""",null,null,"""Bangladesh""","""BGD""","""Southern Asia""","""Asia""","""Khulna, Chittagong""",null,null,null,"""Yes""",null,null,null,null,"""Kph""",null,null,null,null,1970,11,12,1970,11,12,300000,null,3648000,null,3648000,null,null,86400,15.001282,null,null,null,null
"""1970-0026-BGD""",1970,26,null,"""Natural""","""Meteorological""","""Storm""",null,null,null,"""Bangladesh""","""BGD""","""Southern Asia""","""Asia""",null,null,null,null,null,null,null,null,null,"""Kph""",null,null,null,null,1970,4,13,1970,4,13,17,null,110,null,110,null,null,null,15.001282,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2021-0449-YEM""",2021,449,"""FL-2021-000110""","""Natural""","""Hydrological""","""Flood""",null,null,null,"""Yemen""","""YEM""","""Western Asia""","""Asia""","""Dhamar, Amran, Al Mahwit, Mari…","""Heavy rains""",null,null,null,null,null,null,null,"""Km2""",null,null,null,null,2021,7,16,2021,8,7,11,null,67980,null,67980,null,null,null,null,null,null,null,null
"""2021-0075-ZAF""",2021,75,null,"""Natural""","""Hydrological""","""Flood""",null,null,null,"""South Africa""","""ZAF""","""Southern Africa""","""Africa""","""Mpumalanga Province, Free Stat…","""Heavy rains""",null,null,null,null,null,null,null,"""Km2""",null,null,null,null,2021,2,1,2021,2,15,31,null,400,null,400,null,null,75000,null,null,"""2707;77311;77312;77313;77315""",null,"""Free State, KwaZulu-Natal, Lim…"
"""2021-0599-COD""",2021,599,"""EP-2021-000138""","""Natural""","""Biological""","""Epidemic""","""Viral disease""",null,"""Meningitis""","""Congo (the Democratic Republic…","""COD""","""Middle Africa""","""Africa""","""Tshopo province""",null,null,null,null,null,"""Yes""",null,null,"""Vaccinated""",null,null,null,null,2021,9,7,2021,9,13,131,301,null,null,301,null,null,null,null,null,null,null,null


In [14]:
regional_freq = disasters.select(
    ["Year", "Region","disasterType"]
    ).group_by(["Year","Region", "disasterType"], maintain_order=True).len()

# group least frequent disaster types into "other category"
least_freq = regional_freq.group_by("disasterType").sum().sort("len")[:6].select("disasterType")
least_freq = least_freq.to_series().to_list()
annual_country_freq = regional_freq.with_columns(disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))

# regroup to ensure there are not multiple others within one region/year
annual_country_freq = annual_country_freq.group_by(["Year","Region", "disasterType"], maintain_order=True).len()

annual_country_freq


Year,Region,disasterType,len
i64,str,str,u32
1970,"""South America""","""Flood""",1
1970,"""Australia and New Zealand""","""Storm""",1
1970,"""Western Africa""","""Flood""",1
1970,"""Southern Asia""","""Storm""",1
1970,"""Southern Asia""","""Flood""",1
…,…,…,…
2021,"""Northern America""","""Flood""",1
2021,"""Middle Africa""","""Other""",1
2021,"""Northern Africa""","""Drought""",1


In [15]:
# consolidate least frequent disaster types into Other category
annual_frequencies = disasters.with_columns(\
    disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))

annual_frequencies = annual_frequencies.select(
    ["Year", "disasterType"]
    ).group_by(["Year", "disasterType"], maintain_order=True).len()

annual_frequencies = annual_frequencies.with_columns(pl.concat_str([pl.col("Year"), pl.col("disasterType")]).alias("compkey"))

annual_frequencies


Year,disasterType,len,compkey
i64,str,u32,str
1970,"""Flood""",31,"""1970Flood"""
1970,"""Storm""",24,"""1970Storm"""
1970,"""Drought""",2,"""1970Drought"""
1970,"""Landslide""",6,"""1970Landslide"""
1970,"""Earthquake""",12,"""1970Earthquake"""
…,…,…,…
2021,"""Landslide""",10,"""2021Landslide"""
2021,"""Earthquake""",19,"""2021Earthquake"""
2021,"""Other""",10,"""2021Other"""


In [22]:
disaster_costs = pl.read_csv("disaster_costs.csv").rename(
        {"Entity" : "disasterType", "Total economic damages" : "economicdamage"})
disaster_costs = disaster_costs.filter(~pl.col("disasterType").str.contains("All disasters"))
print(least_freq)
disaster_costs = disaster_costs.with_columns(disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))
disaster_costs = disaster_costs.filter(pl.col("Year") >= 1970).filter(pl.col("Year") <= 2021)
disaster_costs = disaster_costs.group_by(["Year", "disasterType"]).sum()


disaster_costs.write_csv("cleaned/disaster_costs.csv")
disaster_costs


['Animal accident', 'Impact', 'Glacial lake outburst', 'Mass movement (dry)', 'Insect infestation', 'Volcanic activity']


Year,disasterType,Code,economicdamage
i64,str,str,i64
2008,"""Drought""",null,290311502
1975,"""Extreme temperature""",null,600000000
2019,"""Earthquake""",null,2409258000
2007,"""Extreme temperature""",null,0
1979,"""Wet mass movement""",null,2466000
…,…,…,…
1994,"""Wildfire""",null,152000000
1998,"""Wildfire""",null,2616208000
1985,"""Wet mass movement""",null,0


In [45]:
full_data = pl.read_csv("full_emdat.csv", ignore_errors=True).filter(pl.col("Start Year") >= 1970).filter(pl.col("Start Year") <= 2021)
disaster_costs = full_data.select(pl.col(["Start Year", "Disaster Type", "Region", "Country", "Total Damage, Adjusted ('000 US$)"]))
disaster_costs = disaster_costs.drop_nulls(subset=pl.col("Total Damage, Adjusted ('000 US$)"))
disaster_costs = disaster_costs.rename(
    {"Disaster Type" : "disasterType", 
     "Start Year" : "Year",
     "Total Damage, Adjusted ('000 US$)" : "Cost ($K)"
    }
)

disaster_costs

Year,disasterType,Region,Country,Cost ($K)
i64,str,str,str,i64
1971,"""Drought""","""Africa""","""Gambia""",1505
1970,"""Flood""","""Africa""","""Morocco""",235442
1970,"""Flood""","""Americas""","""Peru""",40810
1970,"""Flood""","""Americas""","""Argentina""",196202
1970,"""Storm""","""Africa""","""Madagascar""",89468
…,…,…,…,…
2021,"""Wildfire""","""Americas""","""United States of America""",3710805
2021,"""Flood""","""Asia""","""Pakistan""",11245
2021,"""Drought""","""Asia""","""China""",3485908
